In [ ]:
import pandas as pd
import numpy as np

# GENERACIÓN DEL RATING

Para la generación del rating se usan varias tablas:


* **Variables_rating**: Esta tabla ha sido creada manualmente a traves de la tabla de resultados del modelo NIR. Contiene la informacion de las variables que usamos para generar el rating que son:

  * VARIABLES CATEGORICAS: Estas variables son el resultado del modelo nir
  
          'external_audit',  '2030_commitment', 'waste_commitment',      'biodiversity_commitment', 'climate_strategy', 'water_commitment',
          'electric_vehicles', 'green_buildings', 'recognized_env_standard',
          'circular_economy','energy_commitment', 'eco_efficiency','food_loss_waste', 'packaging', 'product_stewardship',
          'circular_fashion', 'building_materials', 'fleet_management',
          'fuel_efficiency', 'descarbonization_strategy', 'ard_management'
       
  * VARIABLES NUMERICAS: Estas variables se consiguen con un calculo con las 
      variables que salen del modelo nir. Se realiza el ratio en %.

          '%_renewable_energy' , '%_recycled_water' ,'%_recycled_waste',
          '%_recycled_haz_waste' ,'%_recycled_nonhaz_waste' , 

* **Tabla_variables_temas**: Esta tabla contiene a que tema pertenecen las variables que utilizaremos para el rating
          
* **Tabla_pesos_tema**: Esta tabla contiene para cada sector cuales son los pesos que le aplica a cada tema. Esta tabla procede de los pesos de materialidad que aplica el Dow Jones. 
**añadir mas cosas, de dodne sale el estudio realizaedo y tal**


* **Tabla_sectores**: Esta tabla contiene cada empresa con el sector al que pertenece



Veamos las variables que vamos a usar par el rating 

In [ ]:
variables_rating=pd.read_csv('variables_rating_app.csv', sep=';')

In [ ]:
variables_rating.head()

,variable,empresa,año,valor
0,external_audit,Cellnex,2020,1.0
1,2030_commitment,Cellnex,2020,1.0
2,waste_management,Cellnex,2020,1.0
3,biodiversity_commitment,Cellnex,2020,1.0
4,climate_strategy,Cellnex,2020,0.0


In [ ]:
variables_rating.variable.unique()

array(['external_audit', '2030_commitment', 'waste_management',
       'biodiversity_commitment', 'climate_strategy', 'water_management',
       'recognized_env_standard', 'energy_management', 'ard_management',
       'green_buildings', 'building_materials', 'resource_efficiency',
       'electric_vehicles', 'packaging', 'fleet_management',
       'fuel_efficiency', 'food_loss_waste', 'decarbonisation_strategy',
       '%_renewable_energy', '%_recycled_waste', '%_recycled_haz_waste',
       '%_recycled_nonhaz_waste', '%_recycled_water', 'circular_economy',
       'product_stewardship', '%_reduction_co2_2030', nan,
       'eco_efficiency', 'circular_fashion'], dtype=object)

Veamos a que tema pertenecen las variables que utilizaremos para calcular el rating

In [ ]:
tabla_temas=pd.read_csv('tabla_variables_temas_app.csv',sep=';')

In [ ]:
tabla_temas[tabla_temas.variable=='water_management']

,tema,variable,type
11,Water Related Risks,water_management,cualitativo


In [ ]:
tabla_temas[tabla_temas.tema=='Decarbonization Strategy']

,tema,variable,type
7,Decarbonization Strategy,decarbonization_strategy,cualitativo


In [ ]:

decarbonization 

Unimos la tabla donde estan los valores que toman las variables con el tema al que pertenece cada una de estas variables

In [ ]:
variables_rating=pd.merge(variables_rating,tabla_temas, on='variable')

In [ ]:
variables_rating=variables_rating.fillna(0)

In [ ]:
variables_rating.dtypes

variable     object
empresa      object
año         float64
valor       float64
tipo         object
tema         object
type         object
dtype: object

In [ ]:
variables_rating['valor'] = variables_rating['valor'].astype(float)

In [ ]:
prueba=variables_rating[variables_rating.empresa=='Mapfre']
prueba[prueba.año==2021]

,variable,empresa,año,valor,tipo,tema,type
282,2030_commitment,Mapfre,2021,1.0,cualitativo,Climate Strategy,cualitativo
581,climate_strategy,Mapfre,2021,1.0,cualitativo,Climate Strategy,cualitativo
895,energy_management,Mapfre,2021,1.0,cualitativo,Operational Eco-Efficiency,cualitativo
1176,decarbonization_strategy,Mapfre,2021,1.0,cualitativo,Decarbonization Strategy,cualitativo
1303,%_renewable_energy,Mapfre,2021,0.0,cuantitativo,Operational Eco-Efficiency,cuantitativo
1494,%_reduction_co2_2030,Mapfre,2021,0.5,cuantitativo,Climate Strategy,cuantitativo


Veamos el porcentaje de variables informadas

In [ ]:
variables_rating['informada'] = variables_rating.apply(
    lambda row: 1 if row.valor>0 else 0 , axis=1)


In [ ]:
total=variables_rating.groupby(['empresa','año']).count().variable.reset_index().rename(columns={"variable": "total"})

In [ ]:
total_cual=variables_rating[variables_rating.type=='cualitativo'].groupby(['empresa','año']).count().variable.reset_index().rename(columns={"variable": "total_cual"})

In [ ]:
variables_rating=pd.merge(variables_rating,total, on=['empresa','año'])
variables_rating=pd.merge(variables_rating,total_cual, on=['empresa','año'])

In [ ]:
porc=variables_rating.groupby(['empresa','año','informada','total','total_cual']).count().type.reset_index().rename(columns={"type": "informadas"})

In [ ]:
porc_cual=variables_rating[variables_rating.type=='cualitativo'].groupby(['empresa','año','informada','total','total_cual']).count().type.reset_index().rename(columns={"type": "informadas_cual"})

In [ ]:
porc=porc.merge(porc_cual,on=['empresa',	'año',	'informada',	'total',	'total_cual'])

In [ ]:
porc.head()

,empresa,año,informada,total,total_cual,informadas,informadas_cual
0,ACS,2017.0,0,11,9,3,2
1,ACS,2017.0,1,11,9,8,7
2,ACS,2018.0,0,11,9,3,1
3,ACS,2018.0,1,11,9,8,8
4,ACS,2019.0,0,11,9,2,1


In [ ]:
porc['%_informadas']=porc['informadas']/porc['total']*100
porc['%_informadas_cualitativas']=porc['informadas_cual']/porc['total_cual']*100

In [ ]:
porc.head()

,empresa,año,informada,total,total_cual,informadas,informadas_cual,%_informadas,%_informadas_cualitativas
1,ACS,2017,1,12,10,9,8,75.000000,80.0
3,ACS,2018,1,12,10,9,9,75.000000,90.0
5,ACS,2019,1,12,10,10,9,83.333333,90.0
7,ACS,2020,1,12,10,7,6,58.333333,60.0
9,ACS,2021,1,12,10,9,8,75.000000,80.0


In [ ]:
porc=porc[porc['informada']==1]

In [ ]:
porc.to_csv('porc_informadas_app_2.csv')

# Calculo del rating

Hay temas que contienen varias variables, para obtener solo un resultado se realiza la media entre estas

In [ ]:
variables_rating.groupby(['tema','empresa','año']).mean().reset_index()

,tema,empresa,año,valor,informada,total,total_cual
0,Biodiversity,ACS,2017.0,1.0,1.0,13.0,11.0
1,Biodiversity,ACS,2018.0,1.0,1.0,13.0,11.0
2,Biodiversity,ACS,2019.0,1.0,1.0,13.0,11.0
3,Biodiversity,ACS,2020.0,1.0,1.0,13.0,11.0
4,Biodiversity,ACS,2021.0,1.0,1.0,13.0,11.0
...,...,...,...,...,...,...,...
1144,Water Related Risks,Unicaja,2017.0,0.0,0.0,8.0,6.0
1145,Water Related Risks,Unicaja,2018.0,1.0,1.0,8.0,6.0
1146,Water Related Risks,Unicaja,2019.0,0.0,0.0,8.0,6.0
1147,Water Related Risks,Unicaja,2020.0,0.0,0.0,8.0,6.0


In [ ]:
variables_rating.tema.unique()

array(['Environmental Reporting', 'Climate Strategy', 'Biodiversity',
       'Environmental Policy & Management Systems',
       'Operational Eco-Efficiency', 'Water Related Risks',
       'Mineral Waste Management',
       'Resource Conservation & Resource Efficiency',
       'Building Materials', 'Packaging', 'Fleet Management',
       'Food Loss & Waste', 'Product Stewardship', 'Circular Fashion'],
      dtype=object)

In [ ]:
variables_rating_final=variables_rating.groupby(['tema','empresa','año']).mean().reset_index()

In [ ]:
prueba=variables_rating_final[variables_rating_final.empresa=='Acciona']
prueba[prueba.año==2021]

,tema,empresa,año,valor,informada,total,total_cual
9,Biodiversity,Acciona,2021.0,1.000000,1.000000,13.0,11.0
126,Building Materials,Acciona,2021.0,1.000000,1.000000,13.0,11.0
161,Climate Strategy,Acciona,2021.0,0.333333,0.333333,13.0,11.0
320,Environmental Policy & Management Systems,Acciona,2021.0,1.000000,1.000000,13.0,11.0
453,Environmental Reporting,Acciona,2021.0,1.000000,1.000000,13.0,11.0
636,Operational Eco-Efficiency,Acciona,2021.0,0.500000,0.500000,13.0,11.0
795,Packaging,Acciona,2021.0,0.000000,0.000000,13.0,11.0
969,Resource Conservation & Resource Efficiency,Acciona,2021.0,0.500000,0.500000,13.0,11.0
999,Water Related Risks,Acciona,2021.0,1.000000,1.000000,13.0,11.0


In [ ]:
variables_rating_final.tema.unique()

array(['Biodiversity', 'Building Materials', 'Circular Fashion',
       'Climate Strategy', 'Decarbonization Strategy',
       'Environmental Policy & Management Systems',
       'Environmental Reporting', 'Fleet Management', 'Food Loss & Waste',
       'Mineral Waste Management', 'Operational Eco-Efficiency',
       'Packaging', 'Product Stewardship',
       'Resource Conservation & Resource Efficiency',
       'Water Related Risks'], dtype=object)

In [ ]:
variables_rating_final.empresa.unique()

array(['ACS', 'Acciona', 'Acciona Energías', 'Acerinox', 'Aena',
       'Amadeus', 'ArcelorMittal', 'Cellnex', 'Colonial', 'Enagas',
       'Endesa', 'Ferrovial', 'Fluidra', 'Grifols', 'IAG', 'Iberdrola',
       'Inditex', 'Indra', 'Logista', 'Melia', 'Merlin Properties',
       'Naturgy', 'Red Eléctrica', 'Repsol', 'Rovi', 'Sacyr', 'Solaria',
       'Telefonica', 'BBVA', 'Banco Sabadell', 'Banco Santander',
       'Bankinter', 'CaixaBank', 'Mapfre', 'Unicaja'], dtype=object)

In [ ]:
variables_rating_cual=variables_rating[variables_rating.type=='cualitativo']

In [ ]:
variables_rating_final_cual=variables_rating_cual.groupby(['tema','empresa','año']).mean().reset_index()

Para teminar y calcular el rating final, se realiza la suma ponderada segun los pesos que tiene cada tema segun el sector. Para ello necesitamos saber cada empresa a que sector pertenece.

In [ ]:
tabla_sectores=pd.read_csv('tabla_sectores_it3.csv',sep=';')

In [ ]:
len(list(tabla_sectores.empresa.unique()))

36

In [ ]:
tabla_sectores[tabla_sectores.empresa=='Colonial']

,empresa,sector
13,Colonial,Real Estate


In [ ]:
variables_rating_final=pd.merge(variables_rating_final,tabla_sectores, on=('empresa'))
variables_rating_final_cual=pd.merge(variables_rating_final_cual,tabla_sectores, on=('empresa'))

In [ ]:
variables_rating_final.tema.unique()

array(['Biodiversity', 'Building Materials', 'Climate Strategy',
       'Environmental Policy & Management Systems',
       'Environmental Reporting', 'Operational Eco-Efficiency',
       'Resource Conservation & Resource Efficiency',
       'Mineral Waste Management', 'Water Related Risks',
       'Fleet Management', 'Packaging', 'Circular Fashion',
       'Product Stewardship', 'Food Loss & Waste'], dtype=object)

In [ ]:
tabla_pesos=pd.read_csv('tabla_pesos_tema.csv', sep=';')

In [ ]:
tabla_pesos[tabla_pesos.sector=='Real Estate']

,peso,tema,sector


In [ ]:
tabla_pesos.sector.unique()

array(['Metals and mining', 'Oil and gas refining', 'Airlines', 'Hotels',
       'Electronic Equipent, Instruments & Components', 'Textiles',
       'Insurance', 'Construction & Engineering', 'Banks',
       'Pharmaceuticals', 'Telecommunication',
       'Energy Equipment & Services', 'Real State'], dtype=object)

In [ ]:
variables_rating_final=pd.merge(variables_rating_final,tabla_pesos, on=('sector','tema'))
variables_rating_final_cual=pd.merge(variables_rating_final_cual,tabla_pesos, on=('sector','tema'))


In [ ]:
variables_rating_final[variables_rating_final.empresa=='Mapfre']

,tema,empresa,año,valor,informada,total,total_cual,sector,peso
723,Climate Strategy,Mapfre,2017.0,0.333333,0.333333,8.0,6.0,Insurance,40.0
724,Climate Strategy,Mapfre,2018.0,0.333333,0.333333,8.0,6.0,Insurance,40.0
725,Climate Strategy,Mapfre,2019.0,0.666667,0.666667,8.0,6.0,Insurance,40.0
726,Climate Strategy,Mapfre,2020.0,0.333333,0.333333,9.0,7.0,Insurance,40.0
727,Climate Strategy,Mapfre,2021.0,0.833333,1.000000,7.0,5.0,Insurance,40.0
728,Operational Eco-Efficiency,Mapfre,2017.0,0.500000,0.500000,8.0,6.0,Insurance,15.0
729,Operational Eco-Efficiency,Mapfre,2018.0,0.790000,1.000000,8.0,6.0,Insurance,15.0
730,Operational Eco-Efficiency,Mapfre,2019.0,0.506655,1.000000,8.0,6.0,Insurance,15.0
731,Operational Eco-Efficiency,Mapfre,2020.0,0.623489,1.000000,9.0,7.0,Insurance,15.0
732,Operational Eco-Efficiency,Mapfre,2021.0,0.500000,0.500000,7.0,5.0,Insurance,15.0


Una vez que tenemos los pesos de cada tema segun la empresa se realiza la media ponderada para calcular el rating. 

In [ ]:
def calculo_rating(empresa,tabla_variables_rating,año):

  datos=tabla_variables_rating[tabla_variables_rating.empresa==empresa]
  datos=datos[datos.año==año]
  score=round(sum(datos['valor']*datos['peso']),2)
  if score>100:
    score=100
  return score

In [ ]:
empresas=list(variables_rating_final.empresa.unique())
años=list(variables_rating_final.año.unique())

In [ ]:
empresas_final=[]
scores=[]
scores_cual=[]
años_final=[]
for i in empresas:
  for j in años:
    score=calculo_rating(empresa=i,tabla_variables_rating=variables_rating_final,año=j)
    scores.append(score)
    score_c=calculo_rating(empresa=i,tabla_variables_rating=variables_rating_final_cual,año=j)
    scores_cual.append(score_c)
    empresas_final.append(i)
    años_final.append(j)

                

In [ ]:
len(empresas_final)

170

In [ ]:
tabla_scores=pd.DataFrame()
tabla_scores['empresas']=empresas_final
tabla_scores['score']=scores
tabla_scores['score_cualitativo']=scores_cual
tabla_scores['año']=años_final

In [ ]:
tabla_scores.head()

,empresas,score,score_cualitativo,año
0,ACS,68.12,68.75,2017
1,ACS,76.56,90.62,2018
2,ACS,87.50,90.62,2019
3,ACS,55.06,62.50,2020
4,ACS,76.56,68.75,2021


In [ ]:
tabla_scores.to_csv('resultados_rating_app_2.csv')
